In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [2]:
users_df = pd.read_csv('MemberData.csv', low_memory=False)
order_df = pd.read_csv('OrderData.csv', low_memory=False)

In [3]:
def tag(starttime, endtime, order_df, users_df):
    order_df_1 = order_df[(order_df['OrderDateTime'] >= starttime) & (order_df['OrderDateTime'] <= endtime)]
    order_df_1 = order_df_1[order_df_1['StatusDef'] == 'Finish']
    order_df_1 = order_df_1.sort_values(by=['ShopMemberId', 'OrderDateTime'])[['ShopMemberId', 'OrderDateTime']]
    order_df_1['OrderDateTime'] = pd.to_datetime(order_df_1['OrderDateTime'])
    order_df1 = pd.concat([order_df_1.rename(columns={'ShopMemberId': 'ShopMemberId1', 'OrderDateTime': 'OrderDateTime1'}), order_df_1.shift()], axis=1)
    order_df1 = order_df1[order_df1['ShopMemberId1'] == order_df1['ShopMemberId']]
    order_df1['period'] = order_df1['OrderDateTime1'] - order_df1['OrderDateTime']
    
    median = order_df1['period'].median()

    merge_df = users_df[users_df["RegisterDateTime"] < endtime][["ShopMemberId"]].merge(order_df[(order_df["StatusDef"] == "Finish") & (order_df["OrderDateTime"] <= endtime)][['ShopMemberId', 'OrderDateTime']], how='left', on='ShopMemberId')
    
    R = merge_df[merge_df["OrderDateTime"].isna()]["ShopMemberId"].values
    R = pd.DataFrame({"ShopMemberId": R, "tag":"R"})
    
    df2 = merge_df[~merge_df["OrderDateTime"].isna()].sort_values(by = ["ShopMemberId", "OrderDateTime"]).drop_duplicates(["ShopMemberId"], keep = "last")[["ShopMemberId", "OrderDateTime"]]
    S = df2[df2["OrderDateTime"] < starttime]["ShopMemberId"].values
    S = pd.DataFrame({"ShopMemberId": S, "tag":"S"})
    
    df_3 = merge_df[(merge_df['OrderDateTime'] >= starttime) & (merge_df['OrderDateTime'] <= endtime)]
    df_4 = df_3.groupby("ShopMemberId", as_index = False).count()
    N = df_4[df_4["OrderDateTime"] == 1]["ShopMemberId"].values
    df_5 = df_3.set_index("ShopMemberId").loc[N].reset_index()
    df_5["OrderDateTime"] = pd.to_datetime(df_5["OrderDateTime"])
    
    y = int(endtime.split("-")[0]); m = int(endtime.split("-")[1]); d = int(endtime.split("-")[2]);
    L = df_5[df_5["OrderDateTime"] < datetime(y, m, d) - median * 3]["ShopMemberId"].values
    N = pd.DataFrame({"ShopMemberId": N, "tag":"N"})
    L = pd.DataFrame({"ShopMemberId": L, "tag":"L"})
    NL = pd.concat([N, L]).drop_duplicates(["ShopMemberId"], keep = "last")
    
    A = df_4[df_4["OrderDateTime"] != 1]["ShopMemberId"].values
    df_6 = df_3.set_index("ShopMemberId").loc[A].reset_index().sort_values(by = ["ShopMemberId", "OrderDateTime"]).drop_duplicates(["ShopMemberId"], keep = "last")
    df_6["OrderDateTime"] = pd.to_datetime(df_6["OrderDateTime"])
    P = df_6[df_6["OrderDateTime"] < datetime(y, m, d) - median * 3]["ShopMemberId"].values

    P = pd.DataFrame({"ShopMemberId": P, "tag":"P"})
    A = pd.DataFrame({"ShopMemberId": A, "tag":"A"})
    PA = pd.concat([A, P]).drop_duplicates(["ShopMemberId"], keep = "last")
        
    return pd.concat([R, S, NL, PA])

In [4]:
a = tag('2021-04-01', '2022-03-31', order_df, users_df)
b = tag('2021-10-01', '2022-09-30', order_df, users_df)
c = tag('2022-04-01', '2023-03-31', order_df, users_df)

merge = a.merge(b, how = "outer", on = "ShopMemberId").merge(c, how = "outer", on = "ShopMemberId")
merge["12"] = merge.apply(lambda x: x.tag_x + x.tag_y if type(x.tag_x) == str and type(x.tag_y) == str else None, axis = 1)
merge["23"] = merge.apply(lambda x: x.tag_y + x.tag if type(x.tag_y) == str and type(x.tag) == str else None, axis = 1)
merge.columns = ["ShopMemberId", "tag1", "tag2", "tag3", "1to2", "2to3"]